In [1]:
from mp_api.client import MPRester
from dotenv import dotenv_values
from ase.io import read
from ase.visualize import view

/home/elcabris/Documentos/scientific computing/chemistry/quimica1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
API_KEY = dotenv_values()['API_KEY']
print(API_KEY)

d6pFCKRaeSdz43y2tmrgcjbBkfU7N4Pj


# Punto 1

In [3]:
bravais_lattice = {
    "P": "Primitive",
    "S": "Base-centered",
    "I": "Body-centered",
    "F": "Face-centered",
    "C": "Base-centered",
    "R": "Primitive"
}

In [4]:
cristal_system = {
    "cubic": None,
    "triclinic": None,
    "orthorhombic": ['mp-558014', 'mp-998864', 'mp-937760', 'mp-973783'],
    "monoclinic": ['mp-557376', 'mp-1196831'],
    "tetragonal": ['mp-1096977', 'mp-632291'],
    "rombohedral": None,
    "hexagonal": ['mp-10851'],
}

In [5]:
with MPRester(api_key=API_KEY) as mpr:
    cristal_system['cubic'] = mpr.summary.search(formula=['W'], crystal_system='Cubic')
    cristal_system['triclinic'] = mpr.summary.search(formula=['S'], crystal_system='Triclinic')
    cristal_system['orthorhombic'] = mpr.summary.search(material_ids=cristal_system['orthorhombic'])
    cristal_system['monoclinic'] = mpr.summary.search(material_ids=cristal_system['monoclinic'])
    cristal_system['tetragonal'] = mpr.summary.search(material_ids=cristal_system['tetragonal'])
    cristal_system['hexagonal'] = mpr.summary.search(material_ids=cristal_system['hexagonal'])
    cristal_system['rombohedral'] = mpr.summary.search(formula=['CaMg(CO3)2'], crystal_system='Trigonal')

/tmp/ipykernel_5466/4142830632.py:2: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  cristal_system['cubic'] = mpr.summary.search(formula=['W'], crystal_system='Cubic')
Retrieving SummaryDoc documents: 100%|█████████| 3/3 [00:00<00:00, 18752.48it/s]
/tmp/ipykernel_5466/4142830632.py:3: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  cristal_system['triclinic'] = mpr.summary.search(formula=['S'], crystal_system='Triclinic')
Retrieving SummaryDoc documents: 100%|██████████| 1/1 [00:00<00:00, 8322.03it/s]
/tmp/ipykernel_5466/4142830632.py:4: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  cristal_system['orthorhombic'] = mpr.summary.search(material_ids=cristal_system['orthorhombic'])
Retrieving SummaryDoc documents: 100%|█████████| 4/4 [00:0

In [6]:
for j in cristal_system.values():
    for i in j:
        information = ''
        information += f'mp-ID: {i.material_id}\n'
        information += f'Fórmula: {i.formula_pretty}\n'
        information += f'Red de Bravais: {i.symmetry.crystal_system} {bravais_lattice[i.symmetry.symbol[0]]}\n'
        information += f'Notación H-M: {i.symmetry.symbol}\n'
        information += f'Grupo espacial: {i.symmetry.number}\n'
        print(information)

mp-ID: mp-11334
Fórmula: W
Red de Bravais: Cubic Primitive
Notación H-M: Pm-3n
Grupo espacial: 223

mp-ID: mp-8641
Fórmula: W
Red de Bravais: Cubic Face-centered
Notación H-M: Fm-3m
Grupo espacial: 225

mp-ID: mp-91
Fórmula: W
Red de Bravais: Cubic Body-centered
Notación H-M: Im-3m
Grupo espacial: 229

mp-ID: mp-561513
Fórmula: S
Red de Bravais: Triclinic Primitive
Notación H-M: P-1
Grupo espacial: 2

mp-ID: mp-558014
Fórmula: S
Red de Bravais: Orthorhombic Primitive
Notación H-M: Pnnm
Grupo espacial: 58

mp-ID: mp-937760
Fórmula: C
Red de Bravais: Orthorhombic Face-centered
Notación H-M: Fmmm
Grupo espacial: 69

mp-ID: mp-998864
Fórmula: Br
Red de Bravais: Orthorhombic Body-centered
Notación H-M: Immm
Grupo espacial: 71

mp-ID: mp-973783
Fórmula: H2
Red de Bravais: Orthorhombic Base-centered
Notación H-M: Cmce
Grupo espacial: 64

mp-ID: mp-557376
Fórmula: S
Red de Bravais: Monoclinic Base-centered
Notación H-M: C2/c
Grupo espacial: 15

mp-ID: mp-1196831
Fórmula: S
Red de Bravais: Mono

# Punto 2

## Celda unitaria y celda primitiva

La celda unitaria es la estructura báisca que se repite en tres dimensiones para formar una red cristalina, esta puede contener uno o más puntos de red en su interior, mientras tanto la celda primitiva es un tipo específico de celda unitaria que contiene **exactamente un punto de red** en su interior
    <img src="celda_unitaria.png" alt="celda unitaria" height="100">
    <img src="celda_primitiva.png" alt="celda primitiva" height="100">

## Contribución fraccionaria
La contribución fraccionaria de cada átomo en la celda unitaria se da mediante la siguiente fórmula:

$$
8 \times \frac{1}{8} = 1
$$

Esto se debe a que es un sistema cúbico primitivo y el átomo está ubicado en la esquina de la celda unitaria.

# Punto 3

In [7]:
def create_cif(mp_summary):
    name = f"{mp_summary.material_id}.cif"
    structure = mpr.get_structure_by_material_id(mp_summary.material_id)
    structure.to(filename=name, fmt="cif")
    return name

def view_structure(mp_summary):
    material = read(create_cif(mp_summary))
    view(material)

def show_all():
    for i in cristal_system.values():
        for j in i:
            view_structure(j)

view_structure(cristal_system['cubic'][1])
# show_all()

Retrieving MaterialsDoc documents: 100%|████████| 1/1 [00:00<00:00, 8507.72it/s]


In [8]:
# contar aniones y cationes
def iones(mp_id):
        structure = mpr.get_structure_by_material_id(mp_id)
        structure.add_oxidation_state_by_guess()
        cations = sum(1 for site in structure if site.specie.oxi_state > 0)
        anions = sum(1 for site in structure if site.specie.oxi_state < 0)
        return cations, anions

In [9]:
for i in cristal_system.values():
    for j in i:
        structure = j.structure.as_dict()['lattice']
        j_iones = iones(j.material_id)
        print(f'{j.material_id}')
        print(f'Red de Bravais: {j.symmetry.crystal_system} {bravais_lattice[j.symmetry.symbol[0]]}')
        print(f'a: {structure['a']}, b: {structure['b']}, c: {structure['c']}')
        print(f'alpha: {structure['alpha']}, beta: {structure['beta']}, gamma: {structure['gamma']}')
        print(f'volumen: {structure['volume']}')
        print(f'cationes: {j_iones[0]}, iones: {j_iones[1]}')
    cristal_system['cubic'][0].structure.as_dict()['lattice']

Retrieving MaterialsDoc documents: 100%|████████| 1/1 [00:00<00:00, 6000.43it/s]


mp-11334
Red de Bravais: Cubic Primitive
a: 5.06227038, b: 5.06227038, c: 5.06227038
alpha: 90.0, beta: 90.0, gamma: 90.0
volumen: 129.7286839630197
cationes: 0, iones: 0


Retrieving MaterialsDoc documents: 100%|████████| 1/1 [00:00<00:00, 6700.17it/s]


mp-8641
Red de Bravais: Cubic Face-centered
a: 2.82275556623129, b: 2.822755247963796, c: 2.82275486
alpha: 59.999993129615255, beta: 59.99999699469613, gamma: 59.99998775979054
volumen: 15.903934900377433
cationes: 0, iones: 0


Retrieving MaterialsDoc documents: 100%|████████| 1/1 [00:00<00:00, 8256.50it/s]


...

mp-1096977
Red de Bravais: Tetragonal Primitive
a: 0.98049292, b: 2.649638738941349, c: 2.649638738941349
alpha: 90.0, beta: 90.0, gamma: 90.0
volumen: 6.883634324939212
cationes: 0, iones: 0


Retrieving MaterialsDoc documents: 100%|████████| 1/1 [00:00<00:00, 8981.38it/s]


mp-6459
Red de Bravais: Trigonal Primitive
a: 6.085409604925538, b: 6.085409604925538, c: 6.0854091445989065
alpha: 47.196259894724754, beta: 47.196259894724754, gamma: 47.196263190687255
volumen: 110.93640357128461
cationes: 4, iones: 6


Retrieving MaterialsDoc documents: 100%|████████| 1/1 [00:00<00:00, 8081.51it/s]

mp-10851
Red de Bravais: Hexagonal Primitive
a: 5.06052279, b: 5.0605216667359585, c: 5.54135331
alpha: 90.0, beta: 90.0, gamma: 119.9999944475218
volumen: 122.89583673299464
cationes: 3, iones: 6


# Punto 4

## Cubico Primitivo (Pm-3n)
- **P:** indica que la celda unidad es primitiva
- **m:** hay plano de reflexión perpendicular a los ejes principales
- **-3:** indica que hay una simetría de rotación de 120° combinada con una reflexión
- **n:** indica que hay un plano de deslizamiento relacionado con la simetría del sistema.

## Cúbico Centrado en las Caras (Fm-3m)
- **F:** indica que la celda unitaria es centrada en las caras
- **m:** representa un plano de reflexión perpendicular a los ejes principales
- **-3:** existe una simetría de rotoinversión, que combina una rotación de 120° con una inversión a través de un punto.
- **m:** indica la presencia de un plano de reflexión relacionado con planos adicionales de simetría en el cristal.

## Cúbico Centrado en el Cuerpo (Im-3m)
- **I:** Indica que la celda unidad es centrada en el cuerpo
- **m:** Representa un plano de reflexión perpendicular a los ejes principales del cristal
- **-3:** Denota una simetría de rotoinversión, que combina una rotación de 120° con una inversión a través de un punto.
- **m:** indica la presencia de un plano de reflexión, relacionado

## Hexágonal Primitivo (P6_422)
- **P:** Indica que la celda unitaria es primitiva
- **6:** Describe un eje de rotación helicoidal con un orden de rotación 6
- **2:** indica que hay un eje de rotación de orden 2 perpendicular al eje de simetría principal del cristal
- **2:** El segundo también indica la presenica de otro eje de rotación de orden 2, pero en una dirección diferente.

## Ortorómbico Primitivo (Pnnm)
- **P:** Indica que la celda unida de es primitiva.
- **nn** La primera "n" representa un plano de deslizamiento perpendicular a uno de los ejes cristalográficos, la segunda "n" representa otro plano de deslizamiento perpendicular a un eje cristalografico diferente.
- **m:** indica la presencia de un plano de relfexión perpendicular al tercer eje cristalográfico.

## Ortorómbico Centrado en la Cara (Fmmm)
- **F:** indica que la celda unidad es centrada en las caras.
- **m:** representa un plano de relfexión perpendicular a uno de los ejes cristalográficos principales.
- **m:** indica la presencia de un plano de reflexión perpendicular a otro de los ejes cristalográficos
- **m:** representa un plano de reflexión perpendicular al tercer eje cristalográfico.

## Ortorómbico Centrado en el Cuerpo (Immm)
- **I:** indica que la celda unidad es centrada en el cuerpo
- **m:** representa un plano de relfexión perpendicular a uno de los ejes cristalográficos principales.
- **m:** indica la presencia de un plano de reflexión perpendicular a otro de los ejes cristalográficos
- **m:** representa un plano de reflexión perpendicular al tercer eje cristalográfico.

## Ortorómbico Centrado en la Base (Cmce)
- **C:** Indica que la celda unidad es centrada en la base.
- **m:** Representa un plano de reflexión prependicular a uno de los ejes cristalográficos principlaes
- **c:** Indica la presencia de un plano de deslizamiento perpendicular a otro de los ejes cristalográficos
- **e:** Representa otro tipo de plano de deslizamiento, pero este es un plano de deslizacmiento doble (

# Punto 5

## Aplicaciones para el Wolframio (W)
- **Acero de alta velocidad:** El wolfraimo se alea con el acero para mejorar su dureza y resistencia al desgaste, lo que lo hace ideal para herramientas de corte como broca, fresas y sierras.

- **Soldadura:** Los electrodods de wolframio se utilizan en la soldadura por arco debido a su alta temperatura de fusión y estabilidad.

## Aplicaciones para el Azufre (S)
- **Fertilizantes:** El azufre se utiliza como fungicida para controlar enfermedades de las plantas causadas por hongos.

- **Producción de ácido sulfúrico:** El uso más importante del azufre es como materia prima para la fabricación de ácido sulfúrico, un compuesto químico fundamental en la producción de fertilizantes, detergentes, plásticos, baterías y muchos otros productos.

*en la tabla tengo 3 redes con el Wolframio y 4 redes con el azufre*